In [26]:
import pyspark
from pyspark.sql import SparkSession
# spark = SparkSession.builder \
#     .master('local') \
#     .appName('myAppName') \
#     .config('spark.executor.memory', '5gb') \
#     .config("spark.cores.max", "6") \
#     .getOrCreate()

StatementMeta(preranasmall, 42, 18, Finished, Available)

In [27]:

DF_Now = spark.sql("SELECT CAST(year(current_timestamp()) AS STRING) AS CurrentYear,CASE WHEN month(current_timestamp()) < 10 THEN CONCAT('0', CAST(month(current_timestamp()) AS STRING)) ELSE CAST(month(current_timestamp()) AS STRING) END AS  CurrentMonth,CASE WHEN day(current_timestamp()) < 10 THEN CONCAT('0', CAST(day(current_timestamp()) AS STRING)) ELSE CAST(day(current_timestamp()) AS STRING) END AS  Today")

CurrentYear = DF_Now[['CurrentYear']].toPandas().loc[0][0]

CurrentMonth = DF_Now[['CurrentMonth']].toPandas().loc[0][0]

Today = DF_Now[['Today']].toPandas().loc[0][0]
 
print(CurrentYear)
 
print(CurrentMonth)

print(Today)

StatementMeta(preranasmall, 42, 19, Finished, Available)

2024
02
22


In [28]:
#Set raw filepath
RawContainerName = 'xyz'
RawStorageAccountName ='preranastorage007'
RawRelativePath = 'raw'

RawADLSPath = 'abfss://%s@%s.dfs.core.windows.net/%s/' % (RawContainerName, RawStorageAccountName, RawRelativePath + "/Year="+CurrentYear +"/Month="+CurrentMonth + "/Day="+ Today)

print(RawADLSPath)

#Read with basePath option
DF_SourceRaw = spark.read.option("basePath", RawADLSPath).parquet(RawADLSPath + "/*.parquet")

# SourceRowCount = DF_SourceRaw.count()
# print(SourceRowCount)

# print(DF_SourceRaw)


StatementMeta(preranasmall, 42, 20, Finished, Available)

abfss://xyz@preranastorage007.dfs.core.windows.net/raw/Year=2024/Month=02/Day=22/


In [29]:
#print(DF_SourceRaw)
#DF_SourceRaw.show(3)

StatementMeta(preranasmall, 42, 21, Finished, Available)

For testing

In [30]:
# #country
# df_dim_country = DF_SourceRaw.select("COUNTRY", "TERRITORY")
# #df_dim_country.show(3)

# #df_dim_country_1 = df_dim_country.groupBy("COUNTRY", "TERRITORY").count()
# df_dim_country = df_dim_country.distinct()
# df_dim_country.show()

# df_dim_country.createOrReplaceTempView("DimCountry_Temp")

# sql_str="select COUNTRY, TERRITORY, ROW_NUMBER() OVER (ORDER BY COUNTRY) as Country_ID from DimCountry_Temp " 


# df_dimCountry = spark.sql(sql_str)
# df_dimCountry.show()


# from pyspark.sql.functions import col

# DF_SourceRaw = DF_SourceRaw.join(df_dimCountry, ((DF_SourceRaw.TERRITORY == df_dimCountry.TERRITORY) & (DF_SourceRaw.COUNTRY == df_dimCountry.COUNTRY)), "leftouter" )\
#     .select(DF_SourceRaw["*"],col("Country_ID"))

# DF_SourceRaw.show(truncate=False)

StatementMeta(preranasmall, 42, 22, Finished, Available)

In [31]:
#country
df_dim_country = DF_SourceRaw.select("COUNTRY", "TERRITORY")
#df_dim_country.show(3)

#df_dim_country_1 = df_dim_country.groupBy("COUNTRY", "TERRITORY").count()
df_dim_country = df_dim_country.distinct()
df_dim_country.show()

df_dim_country.createOrReplaceTempView("DimCountry_Temp")

sql_str="select COUNTRY, TERRITORY, uuid()  as Country_ID from DimCountry_Temp " 


df_dimCountry = spark.sql(sql_str)
df_dimCountry.show()


from pyspark.sql.functions import col

DF_SourceRaw = DF_SourceRaw.join(df_dimCountry, ((DF_SourceRaw.TERRITORY == df_dimCountry.TERRITORY) & (DF_SourceRaw.COUNTRY == df_dimCountry.COUNTRY)), "leftouter" )\
    .select(DF_SourceRaw["*"],col("Country_ID"))

DF_SourceRaw.show(truncate=False)

StatementMeta(preranasmall, 42, 23, Finished, Available)

+-----------+---------+
|    COUNTRY|TERRITORY|
+-----------+---------+
|     Canada|       NA|
|  Singapore|     APAC|
|    Ireland|     EMEA|
|      Spain|     EMEA|
|      Italy|     EMEA|
|  Australia|     APAC|
|    Germany|     EMEA|
|     France|     EMEA|
|        USA|       NA|
|    Belgium|     EMEA|
|     Norway|     EMEA|
|Philippines|    Japan|
|    Finland|     EMEA|
|         UK|     EMEA|
|  Singapore|    Japan|
|    Denmark|     EMEA|
|     Sweden|     EMEA|
|    Austria|     EMEA|
|      Japan|    Japan|
|Switzerland|     EMEA|
+-----------+---------+

+-----------+---------+--------------------+
|    COUNTRY|TERRITORY|          Country_ID|
+-----------+---------+--------------------+
|     Canada|       NA|7051064e-d8c1-418...|
|  Singapore|     APAC|6548a325-f0e7-42c...|
|    Ireland|     EMEA|6fef9ddd-5985-416...|
|      Spain|     EMEA|3d36b476-db36-41a...|
|      Italy|     EMEA|fbf09888-5e06-4fd...|
|  Australia|     APAC|3c89688b-38d1-4b6...|
|    Germany|     

In [32]:
#STATUS 
df_dim_STATUS = DF_SourceRaw.select("STATUS")
#df_dim_STATUS.show(3)

df_dim_STATUS = df_dim_STATUS.distinct()
df_dim_STATUS.show()

df_dim_STATUS.createOrReplaceTempView("DimStatus_Temp")

sql_str="select STATUS, uuid() as Status_ID from DimStatus_Temp " 


df_dim_STATUS = spark.sql(sql_str)
df_dim_STATUS.show()


from pyspark.sql.functions import col

DF_SourceRaw = DF_SourceRaw.join(df_dim_STATUS, (DF_SourceRaw.STATUS == df_dim_STATUS.STATUS),"leftouter" )\
    .select(DF_SourceRaw["*"],col("Status_ID"))
    
DF_SourceRaw.show(truncate=False)

StatementMeta(preranasmall, 42, 24, Finished, Available)

+----------+
|    STATUS|
+----------+
|   Shipped|
|   On Hold|
| Cancelled|
|  Resolved|
|In Process|
|  Disputed|
+----------+

+----------+--------------------+
|    STATUS|           Status_ID|
+----------+--------------------+
|   Shipped|32ec41e1-8a79-43c...|
|   On Hold|9d0e360c-63da-47c...|
| Cancelled|22e962e8-d885-4a5...|
|  Resolved|3fa9b352-1c69-439...|
|In Process|56f2db35-0cb6-405...|
|  Disputed|b402b910-22bd-432...|
+----------+--------------------+

+-----------+---------------+---------+---------------+-------+----------------+-------+------+--------+-------+-----------+----+-----------+--------------------------+----------------+-----------------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+----------------------------+------------------------------------+------------------------------------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|SALES  |ORDERDATE       |STATUS |QTR_ID|MONTH

In [33]:
#DEALSIZE 
df_dim_DEALSIZE = DF_SourceRaw.select("DEALSIZE")
#df_dim_DEALSIZE.show(3)

df_dim_DEALSIZE = df_dim_DEALSIZE.distinct()
df_dim_DEALSIZE.show()

df_dim_DEALSIZE.createOrReplaceTempView("DimDealsize_Temp")

sql_str="select DEALSIZE, uuid() as DEALSIZE_ID from DimDealsize_Temp " 


df_dim_DEALSIZE = spark.sql(sql_str)
df_dim_DEALSIZE.show()


from pyspark.sql.functions import col

DF_SourceRaw = DF_SourceRaw.join(df_dim_DEALSIZE, (DF_SourceRaw.DEALSIZE == df_dim_DEALSIZE.DEALSIZE),"leftouter" )\
    .select(DF_SourceRaw["*"],col("DEALSIZE_ID"))
    
DF_SourceRaw.show(truncate=False)

StatementMeta(preranasmall, 42, 25, Finished, Available)

+--------+
|DEALSIZE|
+--------+
|  Medium|
|   Small|
|   Large|
+--------+

+--------+--------------------+
|DEALSIZE|         DEALSIZE_ID|
+--------+--------------------+
|  Medium|4b08593c-fdae-476...|
|   Small|b0701106-15e6-483...|
|   Large|037f7df5-80e2-4dc...|
+--------+--------------------+

+-----------+---------------+---------+---------------+-------+----------------+-------+------+--------+-------+-----------+----+-----------+--------------------------+----------------+-----------------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+----------------------------+------------------------------------+------------------------------------+------------------------------------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|SALES  |ORDERDATE       |STATUS |QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|CUSTOMERNAME              |PHONE           |ADDRESSLINE1                 |ADDRESSLINE2|CITY

In [34]:
#PRODUCTLINE 
df_dim_PRODUCTLINE = DF_SourceRaw.select("PRODUCTLINE")
#df_dim_PRODUCTLINE.show(3)

df_dim_PRODUCTLINE = df_dim_PRODUCTLINE.distinct()
df_dim_PRODUCTLINE.show()

df_dim_PRODUCTLINE.createOrReplaceTempView("DimProductline_Temp")

sql_str="select PRODUCTLINE, uuid() as PRODUCTLINE_ID from DimProductline_Temp " 


df_dim_PRODUCTLINE = spark.sql(sql_str)
df_dim_PRODUCTLINE.show()


from pyspark.sql.functions import col

DF_SourceRaw = DF_SourceRaw.join(df_dim_PRODUCTLINE, (DF_SourceRaw.PRODUCTLINE == df_dim_PRODUCTLINE.PRODUCTLINE),"leftouter" )\
    .select(DF_SourceRaw["*"],col("PRODUCTLINE_ID"))
    
DF_SourceRaw.show(truncate=False)

StatementMeta(preranasmall, 42, 26, Finished, Available)

+----------------+
|     PRODUCTLINE|
+----------------+
|     Motorcycles|
|    Vintage Cars|
|           Ships|
|Trucks and Buses|
|    Classic Cars|
|          Trains|
|          Planes|
+----------------+

+----------------+--------------------+
|     PRODUCTLINE|      PRODUCTLINE_ID|
+----------------+--------------------+
|     Motorcycles|db357add-6cbc-41d...|
|    Vintage Cars|02844dc3-2f46-4b2...|
|           Ships|833b6adf-2e08-48b...|
|Trucks and Buses|10c15d17-771c-491...|
|    Classic Cars|0cb8d216-315a-492...|
|          Trains|65cc0798-2129-43f...|
|          Planes|40b8b396-fcd3-42f...|
+----------------+--------------------+

+-----------+---------------+---------+---------------+-------+----------------+-------+------+--------+-------+-----------+----+-----------+--------------------------+----------------+-----------------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+-------------------------

In [35]:
#CUSTOMER
df_dim_CUSTOMER = DF_SourceRaw.select("CUSTOMERNAME", "PHONE","ADDRESSLINE1", "ADDRESSLINE2", "CITY", "STATE", "POSTALCODE", "CONTACTLASTNAME", "CONTACTFIRSTNAME")
#df_dim_CUSTOMER.show(3)

df_dim_CUSTOMER = df_dim_CUSTOMER.distinct()
df_dim_CUSTOMER.show(3)

df_dim_CUSTOMER.createOrReplaceTempView("DimCustomer_Temp")

sql_str="select CUSTOMERNAME, PHONE,ADDRESSLINE1, ADDRESSLINE2, CITY, STATE, POSTALCODE, CONTACTLASTNAME,CONTACTFIRSTNAME, uuid() as Customer_ID from DimCustomer_Temp" 


df_dim_CUSTOMER = spark.sql(sql_str)
df_dim_CUSTOMER.show(3)


from pyspark.sql.functions import col
DF_SourceRaw = DF_SourceRaw.join(df_dim_CUSTOMER, (DF_SourceRaw.CUSTOMERNAME == df_dim_CUSTOMER.CUSTOMERNAME),"leftouter" )\
    .select(DF_SourceRaw["*"],col("Customer_ID"))
    

DF_SourceRaw.show(truncate=False)

StatementMeta(preranasmall, 42, 27, Finished, Available)

+--------------------+-------------+--------------------+------------+---------+-----+----------+---------------+----------------+
|        CUSTOMERNAME|        PHONE|        ADDRESSLINE1|ADDRESSLINE2|     CITY|STATE|POSTALCODE|CONTACTLASTNAME|CONTACTFIRSTNAME|
+--------------------+-------------+--------------------+------------+---------+-----+----------+---------------+----------------+
|Cambridge Collect...|   6175555555|      4658 Baden Av.|        null|Cambridge|   MA|   51247.0|          Tseng|           Kyung|
|   Atelier graphique|   40.32.2555|      54, rue Royale|        null|   Nantes| null|   44000.0|        Schmitt|          Carine|
|Norway Gifts By M...|+47 2212 1555|Drammensveien 126...|        null|     Oslo| null|      null|        Klaeboe|             Jan|
+--------------------+-------------+--------------------+------------+---------+-----+----------+---------------+----------------+
only showing top 3 rows

+--------------------+-------------+--------------------+-

In [36]:
#DF_SourceRaw.dtypes

StatementMeta(preranasmall, 42, 28, Finished, Available)

In [37]:
# from pyspark.sql.functions import expr
# DF_SourceRaw1 = DF_SourceRaw.withColumn('ORDERDATE1', expr("to_date(ORDERDATE, 'yyyyMMdd')"))
# #expr("to_date(value, format)"))
# DF_SourceRaw1.show(5)


StatementMeta(preranasmall, 42, 29, Finished, Available)

In [38]:
#PRODUCTCODE 
df_dim_PRODUCTCODE = DF_SourceRaw.select("PRODUCTCODE", "MSRP")
#df_dim_PRODUCTCODE.show(3)

df_dim_PRODUCTCODE = df_dim_PRODUCTCODE.distinct()
df_dim_PRODUCTCODE.show()

df_dim_PRODUCTCODE.createOrReplaceTempView("DimProductcode_Temp")

sql_str="select PRODUCTCODE, uuid() as Productcode_ID from DimProductcode_Temp " 


df_dim_PRODUCTCODE = spark.sql(sql_str)
df_dim_PRODUCTCODE.show()


from pyspark.sql.functions import col

DF_SourceRaw = DF_SourceRaw.join(df_dim_PRODUCTCODE, (DF_SourceRaw.PRODUCTCODE == df_dim_PRODUCTCODE.PRODUCTCODE),"leftouter" )\
    .select(DF_SourceRaw["*"],col("Productcode_ID"))
    
DF_SourceRaw.show(truncate=False)

StatementMeta(preranasmall, 42, 30, Finished, Available)

+-----------+-----+
|PRODUCTCODE| MSRP|
+-----------+-----+
|   S18_1749|170.0|
|   S18_3140|136.0|
|   S12_3990| 79.0|
|   S32_2206| 40.0|
|   S18_4027|143.0|
|   S32_3522| 64.0|
|   S24_2300|127.0|
|  S700_2610| 72.0|
|  S700_2047| 90.0|
|   S18_3320| 99.0|
|   S32_1374| 99.0|
|   S18_4721|148.0|
|   S18_4668| 50.0|
|   S24_2972| 37.0|
|   S18_4522| 87.0|
|   S24_1046| 73.0|
|   S24_1628| 50.0|
|   S24_2841| 68.0|
|   S32_4485|102.0|
|   S18_3782| 62.0|
+-----------+-----+
only showing top 20 rows

+-----------+--------------------+
|PRODUCTCODE|      Productcode_ID|
+-----------+--------------------+
|   S18_1749|4b6f98f3-5a64-456...|
|   S18_3140|9b095481-8d6d-42a...|
|   S12_3990|14c7d43e-ebed-414...|
|   S32_2206|e5e11210-d67b-480...|
|   S18_4027|c767bebe-a62f-433...|
|   S32_3522|390e20b6-69a1-4d3...|
|   S24_2300|25f41ffa-0633-49c...|
|  S700_2610|95e180e4-b4b1-49a...|
|  S700_2047|cbcd2bd8-8f28-4b3...|
|   S18_3320|6cf749e3-0bfc-446...|
|   S32_1374|ac2be4b4-6a7a-4b5...|
|   

In [39]:
df_Fact_Table = DF_SourceRaw.drop('STATUS','QTR_ID','MONTH_ID','YEAR_ID','PRODUCTLINE','MSRP','PRODUCTCODE','CUSTOMERNAME','PHONE','ADDRESSLINE1','ADDRESSLINE2','CITY','STATE','POSTALCODE','COUNTRY','TERRITORY','CONTACTLASTNAME','CONTACTFIRSTNAME','DEALSIZE')


df_Fact_Table.show(truncate=False)

StatementMeta(preranasmall, 42, 31, Finished, Available)

+-----------+---------------+---------+---------------+-------+----------------+----------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|SALES  |ORDERDATE       |DateIngestedDateTime        |Country_ID                          |Status_ID                           |DEALSIZE_ID                         |PRODUCTLINE_ID                      |Customer_ID                         |Productcode_ID                      |
+-----------+---------------+---------+---------------+-------+----------------+----------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+
|101

In [41]:
CuratedContainerName = "xyz"

RawContainerName = 'xyz'
StorageAccountName = RawStorageAccountName
CuratedFolderNameSink = 'curated/DimProductCode'

from delta.tables import *
from pyspark.sql.functions import *
 


CuratedADLSPath = 'abfss://%s@%s.dfs.core.windows.net/%s' % (CuratedContainerName, StorageAccountName, CuratedFolderNameSink)
print(CuratedADLSPath)
print("full load")
df_dim_PRODUCTCODE.write.format("delta")\
    .mode("overwrite")\
    .option("mergeSchema", "false")\
    .save(CuratedADLSPath)

StatementMeta(preranasmall, 42, 33, Finished, Available)

abfss://casestudy@preranastorage007.dfs.core.windows.net/curated/DimProductCode
full load


In [42]:
CuratedContainerName = "xyz"

RawContainerName = 'xyz'
StorageAccountName = RawStorageAccountName
CuratedFolderNameSink = 'curated/DimCUSTOMER'

from delta.tables import *
from pyspark.sql.functions import *
 


CuratedADLSPath = 'abfss://%s@%s.dfs.core.windows.net/%s' % (CuratedContainerName, StorageAccountName, CuratedFolderNameSink)
print(CuratedADLSPath)
print("full load")
df_dim_CUSTOMER.write.format("delta")\
    .mode("overwrite")\
    .option("mergeSchema", "false")\
    .save(CuratedADLSPath)

StatementMeta(preranasmall, 42, 34, Finished, Available)

abfss://xyz@preranastorage007.dfs.core.windows.net/curated/DimCUSTOMER
full load


In [43]:
CuratedContainerName = "xyz"

RawContainerName = 'xyz'
StorageAccountName = RawStorageAccountName
CuratedFolderNameSink = 'curated/DimPRODUCTLINE'

from delta.tables import *
from pyspark.sql.functions import *
 


CuratedADLSPath = 'abfss://%s@%s.dfs.core.windows.net/%s' % (CuratedContainerName, StorageAccountName, CuratedFolderNameSink)
print(CuratedADLSPath)
print("full load")
df_dim_PRODUCTLINE.write.format("delta")\
    .mode("overwrite")\
    .option("mergeSchema", "false")\
    .save(CuratedADLSPath)

StatementMeta(preranasmall, 42, 35, Finished, Available)

abfss://xyz@preranastorage007.dfs.core.windows.net/curated/DimPRODUCTLINE
full load


In [44]:
CuratedContainerName = "xyz"

RawContainerName = 'xyz'
StorageAccountName = RawStorageAccountName
CuratedFolderNameSink = 'curated/DimDEALSIZE'

from delta.tables import *
from pyspark.sql.functions import *
 


CuratedADLSPath = 'abfss://%s@%s.dfs.core.windows.net/%s' % (CuratedContainerName, StorageAccountName, CuratedFolderNameSink)
print(CuratedADLSPath)
print("full load")
df_dim_DEALSIZE.write.format("delta")\
    .mode("overwrite")\
    .option("mergeSchema", "false")\
    .save(CuratedADLSPath)

StatementMeta(preranasmall, 42, 36, Finished, Available)

abfss://casestudy@preranastorage007.dfs.core.windows.net/curated/DimDEALSIZE
full load


In [45]:
CuratedContainerName = "xyz"

RawContainerName = 'xyz'
StorageAccountName = RawStorageAccountName
CuratedFolderNameSink = 'curated/DimSTATUS'

from delta.tables import *
from pyspark.sql.functions import *
 


CuratedADLSPath = 'abfss://%s@%s.dfs.core.windows.net/%s' % (CuratedContainerName, StorageAccountName, CuratedFolderNameSink)
print(CuratedADLSPath)
print("full load")
df_dim_STATUS.write.format("delta")\
    .mode("overwrite")\
    .option("mergeSchema", "false")\
    .save(CuratedADLSPath)

StatementMeta(preranasmall, 42, 37, Finished, Available)

abfss://xyz@preranastorage007.dfs.core.windows.net/curated/DimSTATUS
full load


In [46]:
CuratedContainerName = "xyz"

RawContainerName = 'xyz'
StorageAccountName = RawStorageAccountName
CuratedFolderNameSink = 'curated/DimCountry'

from delta.tables import *
from pyspark.sql.functions import *
 


CuratedADLSPath = 'abfss://%s@%s.dfs.core.windows.net/%s' % (CuratedContainerName, StorageAccountName, CuratedFolderNameSink)
print(CuratedADLSPath)
print("full load")
df_dimCountry.write.format("delta")\
    .mode("overwrite")\
    .option("mergeSchema", "false")\
    .save(CuratedADLSPath)

StatementMeta(preranasmall, 42, 38, Finished, Available)

abfss://casestudy@preranastorage007.dfs.core.windows.net/curated/DimCountry
full load


In [47]:
CuratedContainerName = "xyz"

RawContainerName = 'xyz'
StorageAccountName = RawStorageAccountName
CuratedFolderNameSink = 'curated/Fact_Table'

from delta.tables import *
from pyspark.sql.functions import *
 


CuratedADLSPath = 'abfss://%s@%s.dfs.core.windows.net/%s' % (CuratedContainerName, StorageAccountName, CuratedFolderNameSink)
print(CuratedADLSPath)
print("full load")
df_Fact_Table.write.format("delta")\
    .mode("overwrite")\
    .option("mergeSchema", "false")\
    .save(CuratedADLSPath)

StatementMeta(preranasmall, 42, 39, Finished, Available)

abfss://casestudy@preranastorage007.dfs.core.windows.net/curated/Fact_Table
full load
